In [12]:
import pypsa
import pandas as pd
import glob

In [2]:
# --------- 1) Scenarios  ----------
SC = 1
pars = {
  1: dict(pv_kw=840,  batt_mwh=0.0, wind_kw=0), # 1: 840 kWp PV, no battery
  2: dict(pv_kw=840,  batt_mwh=0.72, wind_kw=0), # 2: +0.72 MWh battery
  3: dict(pv_kw=2000, batt_mwh=2.5,  wind_kw=0), # 3: 2.0 MW PV + 2.5 MWh battery
  4: dict(pv_kw=3900, batt_mwh=8.9,  wind_kw=0), # 4: 3.9 MW PV + 8.9 MWh battery
  5: dict(pv_kw=1300, batt_mwh=4.9,  wind_kw=1300), # 5: 1.3 MW PV + 1.3 MW wind + 4.9 MWh battery
}[SC]

wd = "/home/cs/Documents/Cola/Master/MS_5-Semester/AU-Viborg" # working directory

In [34]:
# Data
ir = 0.07 # interestrate
data = pd.DataFrame({
    "capex": [400, 100, 1100, 400, 500, 8], 
    "FOM": [10, 12, 14, 14, 10, 0], # fixed O&M
    "opex": [0, 0, 0.05, 0.17, 0, 0],
    "lifetime": [35, 35, 27, 20, 20, 20],
}, index=["PVFree", "PVRoof", "onwind", "CHP", "flow_storage", "thermal_storage"]
)
# add O&M to capex:
#data["capex"] = data["capex"]*(ir*(1+ir)**data["lifetime"]/((1+ir)**data["lifetime"]-1)) + data["FOM"]

# Load timeseries
L_e = 0 # Electrical load
L_h = 0 # Heat Load

# Wind timeseries
wind_ts = pd.read_csv(f"{wd}/data/Wind_Vestas-V90-2000_2019.csv", comment="#")
wind_ts["time"] = pd.to_datetime(wind_ts["time"])
wind_ts = wind_ts.drop(columns=["local_time"])
wind_ts["CF"] = wind_ts["electricity"]/2000 # 2000 rated power

# Solar timeseries
# solar_kWp = [2003, 1056, 6643, 731, 2003, 1056, 6643, 731]
# solar_locs = ["SE", "S", "SW", "E", "NW", "N", "NE", "W"]
solar_dict = {'SE': 2003, 'S': 1056, 'SW': 6643, 'E': 731, 'NW': 2003, 'N': 1056, 'NE': 6643, 'W': 731}
solar_ts = {}
for loc in solar_dict:  # loop directly over dictionary keys
    files = glob.glob(f"{wd}/data/Solar_{loc}*.csv")
    if not files:
        continue
    ts = pd.read_csv(files[0], skiprows=10, skipfooter=11, engine="python")
    ts["time"] = pd.to_datetime(ts["time"].str.replace(":", ""), format="%Y%m%d%H%M")
    ts["time"] = ts["time"].dt.floor("h")
    # Compute capacity factor using the dictionary
    solar_ts[loc] = ts["P"] / (solar_dict[loc] * 1000)

In [ ]:
# --------- 2) Network ----------
n = pypsa.Network()
n.set_snapshots(snapshots)
n.add("Bus","elec"); n.add("Bus","heat"); n.add("Bus","gas")

n.add("Load","L_e", bus="elec", p_set=el/1e3)      # kW
n.add("Load","L_h", bus="heat", p_set=heat/1e3)

# CHP: gas -> electricity (40%) + heat (50%), fixed 635 kWel
n.add("Link","CHP",
      bus0="gas", bus1="elec", bus2="heat",
      p_nom=1587.5, # 635kw_el/0.4
      efficiency=0.40, efficiency2=0.50,
      capital_cost=data.at["CHP", "capex"],
      marginal_cost=data.at["CHP", "opex"])

# Biogas supply price (€/kWh -> €/MWh)
n.add("Carrier","biogas")
n.add(
    "Generator",
    "biogas_supply",
    bus="gas",
    carrier="biogas",
    p_nom=1e6,
    marginal_cost=0)

# add rooftop solar
for loc in solar_dict:
    n.add(
        "Generator",
        f"solar_{loc}",
        bus="elec",
        carrier="solar",
        p_max_pu=solar_ts[f"loc"],
        capital_cost=data.at["capex", "PVRoof"],
        p_nom_extendable=True,
        p_nom_max = solar_dict[loc]
    )
# add freefield solar
n.add(
    "Generator",
    "solar_S_Free",
    bus="elec",
    carrier="solar",
    p_max_pu=solar_ts["S"], 
    capital_cost=data.at["capex", "PVFree"],
    p_nom_extendable=True,
    p_nom_max = 4000 # own estimate
)

# add windpower
n.add(
    "Generator",
    "wind",
    bus="elec",
    carrier="wind",
    p_max_pu=wind_ts["CF"],
    capital_cost=data.at["capex", "wind"],
    p_nom_extendable=True,
    p_nom_max = 2000 # own estimate = std windturbine
)

